# Crawler for ATM location Data

In [172]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup

In [173]:
keywords = ['충청남도 현금인출기', '대전광역시 현금인출기', '세종특별자치시 현금인출기']
options = Options()
options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.

In [171]:
def crawl(keyword) : 
    # Driver execute
    driver = webdriver.Chrome('/Users/dangchan/Documents/chromedriver')
    driver.get("https://map.naver.com/v5/search")
    try:
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/app/layout/dynamic-content-outlet/promotion-popup/div/button"))
    ) #입력창이 뜰 때까지 대기
    finally:
        pass

    # popup close
    driver.find_element(by=By.XPATH, value= "/html/body/app/layout/dynamic-content-outlet/promotion-popup/div/button").click()

    # search box
    search_box = driver.find_element(by=By.CSS_SELECTOR, value="div.input_box>input.input_search")
    search_box.send_keys("충청남도 현금지급기") # 검색어 여기서 설정
    time.sleep(2)
    search_box.send_keys(Keys.ENTER)
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'iframe#searchIframe')))

    # Iframe Switch
    frame = driver.find_element(by=By.CSS_SELECTOR, value="iframe#searchIframe")

    driver.switch_to.frame(frame)

    scroll_div = driver.find_element(By.XPATH, "/html/body/div[3]/div/div[2]/div[1]")

    # Scroll bar > 전체내용 표시되게끔
    driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
    time.sleep(1)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
    time.sleep(1)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
    time.sleep(1)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
    time.sleep(1)
    driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
    time.sleep(1)

    result = []
    page = 1

    while True:
        stores_box = driver.find_element(By.XPATH,"/html/body/div[3]/div/div[2]/div[1]/ul")
        stores = stores_box.find_elements(By.CSS_SELECTOR, 'div.ouxiq.icT4K')
    
        for i,s in enumerate(stores):
            name = s.find_element(By.CSS_SELECTOR, 'span.place_bluelink.YwYLL').text
            expand = s.find_element(By.CSS_SELECTOR, 'span.JXQZb')
            expand.click()
            time.sleep(0.5)
            address = s.find_element(By.CSS_SELECTOR, 'div.o8CtQ').text[3:-2]
            result.append([name, address])

        page += 1

        try:
            next_button = driver.find_element(By.LINK_TEXT, str(page))
            next_button.click()
            time.sleep(5)
        except:

            break

    driver.close()

    # 저장
    result = pd.DataFrame(result, columns = ['NAME', 'ADD'])
    result.to_csv(f'data/{keyword}.csv', index=False)

In [ ]:
for keyword in keywords:
    crawl(keyword=keyword)
    print(f'Crawling finished : {keyword}')